In [1]:
import getpass
from arcgis.gis import GIS

print('Modules Loaded')

Modules Loaded


In [2]:
# Login to ArcGIS Online (AGOL) by requesting the user to input a username and password
# documentation on getpass: https://docs.python.org/3/library/getpass.html

def login_AGOL():
    # prompt the user to enter their AGOL username
    username = input("Enter your ArcGIS Online username:")
    
    # prompt the user to enter their AGOL password (without echoing)
    password = getpass.getpass("Enter your ArcGIS Onine password:")
    
    # create a GIS object and login to AGOL
    gis = GIS("https://www.arcgis.com", username, password)
    
    # return the object
    return gis

# run module
gis = login_AGOL()

# confirm user login 
print("Logged in as:", gis.users.me.username)

Enter your ArcGIS Online username:stacey_vernon_build
Enter your ArcGIS Onine password:········
Logged in as: stacey_vernon_build


In [9]:
# Utilise the AGOL API and the a "static" GeoJSON file to publish data to a feature service. Specifically publish
# Dublin Bike zone data.
# documentation on using the ArcGIS API to create a feature service from GeoJSON here: https://developers.arcgis.com/python/guide/import-data/
# We will determine if a feature layer exists, if not we will create one but if it does - we skip this function

# generate a list of the below search conditions in our AGOL Item Content
zoneSearch = gis.content.search("title:BikeStations_Zones",item_type="Feature Layer")

# if list is empty, create a new feature layer
if not zoneSearch:

# set up the feature service properties in AGOL for the stations and zone dataset
    item_properties = {
    "title": "BikeStations_Zones", #provide a title for your item
    "description": "Dublin Bike Stations and Zones generated from GeoJSON provided by Bleeper Bikes", #add a description of the data
    "tags": "dublinBikes", # add tags to data to improve AGOL search functionality
    "type": "GeoJSON" # define the type of data you are trying to pubish
    }

    # URL link to the GeoJSON file
    geojson_url = "https://data.smartdublin.ie/dataset/09870e46-26a3-4dc2-b632-4d1fba5092f9/resource/40a718a8-cb99-468d-962b-af4fed4b0def/download/bleeperbike_map.geojson"
    geojson_item = gis.content.add(item_properties, geojson_url)

    # publish the GeoJSON as a feature layer
    BikeStationZone_data = geojson_item.publish()
    print("Data published. New feature layer created.")
else:
    print("Bike Station and Zone data already exists in AGOL.")

Bike Station and Zone data already exists in AGOL.


In [ ]:
# need to do:
# improve commenting 
# add in docstrings
# publish live bike data
# get live bike data to refresh
# display in webmap
# product heatmap of current locations - deduce busy areas??